# Vytvoření vlastního obrazu

## Co se naučíte?

* vytvářet vlastní obrazy
* využívat nástroj Docker Compose



Vytvoření vlastního obrazu je v případě Dockeru relativně snadné a spočívá ve vytvoření souboru `Dickerfile` popisujícího obraz a následné vytvoření obrazu pomocí volání `docker build`.

## Vytváření obrazu pro IPython

Předokládejme, že chcete pro přítele  vytvořit jednoduchý obraz, který by mu umožnil snadné spouštění IPythonu (interaktivní REPL, z něhož je odvozen Jupyter, vystačí si však jen s konzolí). Navíc bychom rádi přidali i nějaký užitečný externí modul např. `requests`.

Poznámka: pro nepřítele bychom zvolili jiný programovací jazyk:)

Obsah souboru Dockerfile:

```dockerfile
FROM alpine

RUN apk add ipython py3-pip
RUN pip install requests

WORKDIR /coder

ENTRYPOINT "/usr/bin/ipython"
```

Příkaz `FROM` určuje obraz, který budeme rozšiřovat. Typicky je to nějaká odlehčená distribuce Linuxu. Zvlášť oblíbeny je *Alpine Linux* s nároky v řádu vyšších jednotek MiB. Alias jména posledního obrazu této distribuce je `alpine`.

Další vrstvu přidá volání příkazu `apk`, což je v Alpine manažer balíčků. Zde přidáme balíčky `ipython` a `py3-pip` a balíčky závislé (bude jich nakonec celých padesát).

Další vrstvu získáme použitím správce Python modulů `pip` (ten jsme si nainstalovali v předchozím kroku).

Tím máme hotovu hlavní část práce a zbývá nám jen vytvoření pracovního adresáře pro případné soubory. To zajistá příkaz `WORKDIR`, který změní pracovní adresář (všechny případné další příkazy budou vykonány v tomto adesáři) a pokud neexistuje, tak ho vytvoří (i s případnými meziadresáři).

Ṕoslední příkaz definuje standardní vstupní bod kontejneru, což je typicky buď spuštění hlavní aplikace (zde je to *ipython*) resp. spuštění shellu.

V vadresáři, kde jste soubor `Dockerfile` vytvořili, nyní stačí spustit příkaz `docker build -t "ipython" .`, kde parametr `-t` určuje jméno obrazu (pokud bude lokální stačí zvolit jednoduché jméno, při případném exportu do repozitáře je nutné dodržovat jisté zásady). Posledním parametrem je pracovní adresář, zde `.` (tečka)

**Otázka**: Proč jsou kolem jména (tagu) obrazu uvozovky?

Nyní je možné spustit a to příkazem `docker run  -it ipython`.

**Úkol**: Zjistěte jakou velikost má námi vytvořený obraz?

**Úkol**: Vytvořte obraz, který poskytuje symbolické výpočty v jazyce Julia (vychozí obraz `julia:alpine`) s přidaným balíkem `Symbolics`. Vyzkoušejte pro derivování :)

Nevýhodou výše uvedeného `Dockerfile` je  malá pružnost, neboť uživatel může požadovat i další externí moduly. Může si je sice doinstalovat i uvnitř IPythonu, ale ty zmizí spolu se zánikem kontejneru (kontejnery nejsou persistentní). Při větším počtu modulů bude seznam za `pip` dlouhý a tak je vhodnější číst jména modulů z externího souboru.

Proto Dockerfile mírně pozměníme.

```dockerfile
FROM alpine

COPY requirements.txt .

RUN apk add ipython py3-pip
RUN pip install -r requirements.txt

WORKDIR /coder

ENTRYPOINT "/usr/bin/ipython"
```

Novinkou je příkaz `COPY`, který při tvorbě obrazu (nikoliv při jeho spuštění) kopíruje soubor z hostitelského souborového systému (první parametr) do umístění v souborovém systému obrazu (zde do aktuálního adresáře, což je v daném místě kořenový adresář).

Soubor `requirements.txt` musí být při budování obrazu ce stejném adresáři jako `Dockerfile` a musí obsahovat jména požadovaných modulů (každý na jednom řádku), např.
```
requests
urllib3
```

### Vytváření obrazu pro Jupyter Notebook

Tento obrat spustí server *Jupyter notebooků* na standartním portu, který bude dostupný i zvenčí.

```dockerfile
FROM python:3.9-alpine

RUN apk add --update --no-cache build-base linux-headers libffi-dev
RUN pip install notebook

COPY entrypoint.sh /

ENTRYPOINT ["/bin/sh",  "/entrypoint.sh"]
EXPOSE 8888
```

Obraz tentokrát vychází z oficiálního obrazu Pythonu 3.9 (v Alpine Linuxu), aby nebylo nutné instalovat relativně rozsáhlý Python. Do Alpine Linuxu je nutno doinstalovat překladač C s knihovnami a hlavičkovými soubory (kromě standardních i ty linuxovské). Navíc je potřeba knihovna `libffi` i s hlavičkovýni soubory (zjištěno empiricky, jinak se podpora pro Jupyter notebook nepřeloží). Přidání podpory notebooku zajistí `pip`. 

Vstupním bodem, je tentokrát skript, který je zkopírován do kořenového adresáře kontejnerového FS. Skript spouští jupyter notebook server (kernel) s několika nezbytnými parametry.

`NotebookApp.token` — při přihlášení není prováděna autentizace tokenem (komplikuje to přihlášení a je to relativně bezpečné, neboť kompromitovatelný by měl být kontejner)

`ip` — server poskytuje své služby na implicitní (vnější) adrese (implicitně je to `localhost` a byl by tudíž viditelný jen uvnitř kontejneru). Port zůstává standardní (tj. 8888).

`no-browser` — nespouští se automaticky prohlížeč (žádný není v kontejneru nainstalován, k serveru se bude přihlašovat prohlížeč v hostitelském OS)

`--allow-root` — kontejner se spouští s právy roota (což by bylo vně kontejneru nebezpečné a tak je to implicitně zakázané)


```bash
jupyter notebook --NotebookApp.token='' --ip 0.0.0.0 --no-browser --allow-root
```

Příkaz `ENTRYPOINT` využívá alternativní syntaxi, v němž jsou jednotlivé části příkazového řádku předány jako prvky seznamu.

Příkaz `EXPOSE` exportuje port síťového rozhraní (zde standardní port Jupyter Notebooku).

Obraz se vybuduje příkazem (není interaktivní, lze spustit i jako detached):

`docker build -t "jupyter" .`
    
a spuští příkazem:
    
`docker run -p 8889:8888 jupyter`

Port serveru je v hostitelském localhostu namapován na port 8889 (původní port je využíván lokálním jupyter server, v rámci něhož vzniká tato opora).

K serveru se pak lze (v hostitelském OS) připojit na adrese *localhost:8889*.

![Jupyter](jupyter.png)

**Otázka:** Jak byste ověřilo, že jste se připojili ke správnému (kontejnerizovanému) notebook serveru?
    
**Úkol:** Vytvořené notebooky nejsou presistentní. Zajistěte aby byly?

Při návrhu obrazu je nutné zohlednit skutečnost, kontejnerizované systémy jsou minimální (neběží v nich ani běžné služby). Například následující Dockerfile nevyprodukuje použitelný obraz.

```dockerfile
FROM ubuntu

RUN apt-get install mc

ENTRYPOINT /bin/sh
```

**Úkol**: Upravte obraz tak aby byl funkční.

## Docker Compose

Jednou z nevýhod Docker obrazů jsou omezené možnosti skládání (komposabilita). Obrazy lze sice snadno rozšiřovat, ale nelze je jednoduše skládat.

Ukažme si to například na požadavku vytvoření vývojového prostředí pro testování pythonského přístupu k databázi Redis. Pro Python i Redis existují oficiální obrazy a nabízejí se tak dvě cesty:

* využít oficiální obraz Pythonu a v něm doinstalovat Redis 
* využít oficiální obraz Redisu a v něm doinstalovat Python

I když jsou obě možnosti použitelné (první je z důvodů výrazně větší komplexity Pythonu proti Redisu vhodnější), mají určité nevýhody:

* v případě využití oficiálního obrazu je aktualiace výrazně snadnější (nezávisí na mechanismu aktualizacé kontejnerového OS)
* může docházet ke kolizím 

Řešení nabízí `docker-compose`, který umožňuje spouštět a především propojovat několik kontejnerů a to prostřednictvím:

1) síťového rozhraní (kontejnery navzájem vidí své porty a jsou dostupné přes jednoduché DNS adresy)
2) mohou sdílet souborové systémy (*volumes*)

Popis vícekontejnerového kompozita obsahuje soubor ve formátu YAML s přímočarou syntaxí.

Ukázka definuje kompositum, jenž vzniká instanciací tří obrazů (za běhu tedy obsahuje tři kontejnery)

1) výše uvedený obraz pro jupyter (pro skutečně užitečné použití je ho nutné trochu rozšířit)
2) oficiální obraz pro NoSQL databázi `Redis`
3) oficiální obraz pro NoSQL databází `Mongo`

```yaml
version: "3.2"
services:
  python:
    image: "jupyter"
    stdin_open: true # docker run -i
    tty: true        # docker run -t
    volumes:
      - ./coder:/home/coder
    ports:
      - "2242:22"
  redis:
    image: "redis:alpine"
  mongodb:
    image: "mongo"
    restart: always
    environment:
      MONGO_INITDB_ROOT_USERNAME: root
      MONGO_INITDB_ROOT_PASSWORD: root
    volumes:
      - ./data:/data/db
```

Jádrem konfiguračního souborů je sekce `services`, která obsahuje podsekce, jež definují jednotlivé kontejnery.
V těchto podsekcích jsou pak uvedeny parametry odpovídající volání `docker run`.

Kontejnery  po spuštění tvoří virtuální síť, v níž jsou jednotlivé kontejnery (v roli hosts) identifikovány jménem příslušné sekce a nabízejí porty uvedené v sekci `EXPOSES` dockerfilu. Mimo tuto síť (v hostitelském localhostu) jsou vidět jen porty mapované v sekci `ports`.

**Úkol** : Doplňte obraz `jupyter`, tak aby byl užitečný v rámci kompozita.

Spuštění všech komponent se děje příkazem:
```
docker-compose up
```
přičem konfigurace musí být uložena v souboru `docker-compose.yml` v aktuálním pracovním adresáři.

Komponenty se ukončují příkazem, `docker-compose down` (kontejnery jsou zároveň odstraněny stejně jako jimi vlastněné prostředky).

Praktické je i zobrazení běžících kontejnerů příkazem `docker-compose ps`.

**Úkol**: Vytvořte kompositum python + flask využívající `nginx` http server a ověřte ho v praxi.